In [1]:
cd /content/drive/My\ Drive/111\ Ethicsbot/topic_modeling/

/content/drive/My Drive/111 Ethicsbot/topic_modeling


In [2]:
ls

'Copy of topic_modeling.ipynb'   Unsupervised-Aspect-Extraction/
 P17-1036.pdf                    Unsupervised-Aspect-Extraction_/
 topic_modeling.ipynb


In [3]:
cd Unsupervised-Aspect-Extraction/

/content/drive/My Drive/111 Ethicsbot/topic_modeling/Unsupervised-Aspect-Extraction


In [4]:
ls

code/        preprocessed_data/     pre_trained_model/
LICENSE.txt  preprocessed_data_new  README.md
__MACOSX/    preprocessed_data.zip  requirements.txt


In [5]:
!pip install -r requirements.txt

     |████████████████████████████████| 337kB 4.7MB/s 
     |████████████████████████████████| 51.1MB 61kB/s 
     |████████████████████████████████| 23.5MB 53.5MB/s 
     |████████████████████████████████| 13.9MB 241kB/s 
     |████████████████████████████████| 573kB 41.9MB/s 
     |████████████████████████████████| 3.3MB 37.9MB/s 
ERROR: xarray 0.15.1 has requirement setuptools>=41.2, but you'll have setuptools 39.1.0 which is incompatible.
ERROR: umap-learn 0.4.3 has requirement numpy>=1.17, but you'll have numpy 1.15.4 which is incompatible.
ERROR: plotnine 0.6.0 has requirement numpy>=1.16.0, but you'll have numpy 1.15.4 which is incompatible.
ERROR: google-auth 1.7.2 has requirement setuptools>=40.3.0, but you'll have setuptools 39.1.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: astropy 4.0.1.post1 has requirement numpy>=1.16, but you'll have numpy 1.15.4 which is incompatible.
ERROR: a

In [25]:
cd /content/drive/My Drive/111 Ethicsbot/topic_modeling/Unsupervised-Aspect-Extraction

/content/drive/My Drive/111 Ethicsbot/topic_modeling/Unsupervised-Aspect-Extraction


In [0]:
# !unzip preprocessed_data.zip

In [26]:
cd preprocessed_data/restaurant/

/content/drive/My Drive/111 Ethicsbot/topic_modeling/Unsupervised-Aspect-Extraction/preprocessed_data/restaurant


In [27]:
ls

test_label.txt  test.txt  train.txt  vocab  w2v_embedding


In [32]:
import pandas as pd

sample_train = pd.read_csv("train.txt", sep="delimiter", header=None)
sample_test = pd.read_csv("test.txt", sep="delimiter", header=None)
sample_test_label = pd.read_csv("test_label.txt", sep="delimiter", header=None)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning 

In [35]:
sample_test.head(), sample_train.head()

(                                                   0
 0  staff friendliest competent stickler service e...
 1  great group great date great early brunch nigh...
 2  like cafe noir dont get wrong jsut people work...
 3  service terrible wait everything ask several d...
 4  waitress seems concerned looking good actually...,
                                                    0
 0  update asshole boyfriend dog put without telli...
 1  update asshole telling daughter give dream rem...
 2  asshole wanting watch wife pleasured someone e...
 3    would asshole wear dress sister wedding removed
 4  update asshole breaking girlfriend rude kid re...)

In [39]:
cd code/

/content/drive/My Drive/111 Ethicsbot/topic_modeling/Unsupervised-Aspect-Extraction/code


In [40]:
#!/usr/bin/env python
#  -*- coding: utf-8  -*-

import codecs
import operator
import re

num_regex = re.compile(r'^[+-]?[0-9]+\.?[0-9]*$')


def is_number(token):
    return bool(num_regex.match(token))


def create_vocab(domain, maxlen=0, vocab_size=0):
    assert domain in {'restaurant', 'beer'}
    source = '../preprocessed_data/' + domain + '/train.txt'

    total_words, unique_words = 0, 0
    word_freqs = {}

    fin = codecs.open(source, 'r', 'utf-8')
    for line in fin:
        words = line.split()
        if maxlen > 0 and len(words) > maxlen:
            continue

        for w in words:
            if not is_number(w):
                try:
                    word_freqs[w] += 1
                except KeyError:
                    unique_words += 1
                    word_freqs[w] = 1
                total_words += 1

    print('   %i total words, %i unique words' % (total_words, unique_words))
    sorted_word_freqs = sorted(word_freqs.items(), key=operator.itemgetter(1), reverse=True)

    vocab = {'<pad>': 0, '<unk>': 1, '<num>': 2}
    index = len(vocab)

    for word, _ in sorted_word_freqs:
        vocab[word] = index
        index += 1
        if vocab_size > 0 and index > vocab_size + 2:
            break
    if vocab_size > 0:
        print('  keep the top %i words' % vocab_size)

    # Write (vocab, frequence) to a txt file
    vocab_file = codecs.open('../preprocessed_data/%s/vocab' % domain, mode='w', encoding='utf8')
    sorted_vocab = sorted(vocab.items(), key=operator.itemgetter(1))
    for word, index in sorted_vocab:
        if index < 3:
            vocab_file.write(word + '\t' + str(0) + '\n')
            continue
        vocab_file.write(word + '\t' + str(word_freqs[word]) + '\n')
    vocab_file.close()

    return vocab


def read_dataset(domain, phase, vocab, maxlen):
    # assert domain in {'restaurant', 'beer'}
    assert phase in {'train', 'test'}

    source = '../preprocessed_data/' + domain + '/' + phase + '.txt'
    num_hit, unk_hit, total = 0., 0., 0.
    maxlen_x = 0
    data_x = []

    fin = codecs.open(source, 'r', 'utf-8')
    for line in fin:
        words = line.strip().split()
        if maxlen > 0 and len(words) > maxlen:
            continue
        if not len(words):
            continue

        indices = []
        for word in words:
            if is_number(word):
                indices.append(vocab['<num>'])
                num_hit += 1
            elif word in vocab:
                indices.append(vocab[word])
            else:
                indices.append(vocab['<unk>'])
                unk_hit += 1
            total += 1

        data_x.append(indices)
        if maxlen_x < len(indices):
            maxlen_x = len(indices)

    print('   <num> hit rate: %.2f%%, <unk> hit rate: %.2f%%' % (100 * num_hit / total, 100 * unk_hit / total))
    return data_x, maxlen_x


def get_data(domain, vocab_size=0, maxlen=0):
    print('Reading data from', domain)
    print(' Creating vocab ...')
    vocab = create_vocab(domain, maxlen, vocab_size)
    print(' Reading dataset ...')
    print('  train set')
    train_x, train_maxlen = read_dataset(domain, 'train', vocab, maxlen)
    print('  test set')
    test_x, test_maxlen = read_dataset(domain, 'test', vocab, maxlen)
    maxlen = max(train_maxlen, test_maxlen)
    return vocab, train_x, test_x, maxlen


if __name__ == "__main__":
    vocab, train_x, test_x, maxlen = get_data('restaurant')
    print(len(train_x))
    print(len(test_x))
    print(maxlen)

Reading data from restaurant
 Creating vocab ...
   4850241 total words, 45223 unique words
 Reading dataset ...
  train set
   <num> hit rate: 0.90%, <unk> hit rate: 0.00%
  test set
   <num> hit rate: 0.41%, <unk> hit rate: 5.14%
53127
1490
1567


# Preprocess data

In [53]:
cd /content/drive/My Drive/111 Ethicsbot/topic_modeling/Unsupervised-Aspect-Extraction_

/content/drive/My Drive/111 Ethicsbot/topic_modeling/Unsupervised-Aspect-Extraction_


In [56]:
ls datasets/

ethicsbot/


In [57]:
ls datasets/ethicsbot/

train.txt


In [60]:
cd /content/drive/My Drive/111 Ethicsbot/topic_modeling/Unsupervised-Aspect-Extraction_

/content/drive/My Drive/111 Ethicsbot/topic_modeling/Unsupervised-Aspect-Extraction_


In [61]:
ls

code/         LICENSE.txt         pre_trained_model/
datasets/     __MACOSX/           README.md
datasets.zip  preprocessed_data/  requirements.txt


In [62]:
cd preprocessed_data

/content/drive/My Drive/111 Ethicsbot/topic_modeling/Unsupervised-Aspect-Extraction_/preprocessed_data


In [67]:
cd ethicsbot/

/content/drive/My Drive/111 Ethicsbot/topic_modeling/Unsupervised-Aspect-Extraction_/preprocessed_data/ethicsbot


In [0]:
ls

In [73]:
cd /content/drive/My Drive/111 Ethicsbot/topic_modeling/Unsupervised-Aspect-Extraction_

/content/drive/My Drive/111 Ethicsbot/topic_modeling/Unsupervised-Aspect-Extraction_


In [74]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [75]:
#!/usr/bin/env python
#  -*- coding: utf-8  -*-

from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import codecs


def parse_sentence(line):
    lmtzr = WordNetLemmatizer()
    stop = stopwords.words('english')
    text_token = CountVectorizer().build_tokenizer()(line.lower())
    text_rmstop = [i for i in text_token if i not in stop]
    text_stem = [lmtzr.lemmatize(w) for w in text_rmstop]
    return text_stem


def preprocess_train(domain):
    f = codecs.open('datasets/' + domain + '/train.txt', 'r', 'utf-8')
    out = codecs.open('preprocessed_data/' + domain + '/train.txt', 'w', 'utf-8')

    for line in f:
        tokens = parse_sentence(line)
        if len(tokens) > 0:
            out.write(' '.join(tokens) + '\n')
        out.write(" \n")


def preprocess_test(domain):
    # For restaurant domain, only keep sentences with single
    # aspect label that in {Food, Staff, Ambience}

    f1 = codecs.open('datasets/' + domain + '/test.txt', 'r', 'utf-8')
    f2 = codecs.open('datasets/' + domain + '/test_label.txt', 'r', 'utf-8')
    out1 = codecs.open('preprocessed_data/' + domain + '/test.txt', 'w', 'utf-8')
    out2 = codecs.open('preprocessed_data/' + domain + '/test_label.txt', 'w', 'utf-8')

    for text, label in zip(f1, f2):
        label = label.strip()
        if domain == 'restaurant' and label not in ['Food', 'Staff', 'Ambience']:
            continue
        tokens = parse_sentence(text)
        if len(tokens) > 0:
            out1.write(' '.join(tokens) + '\n')
            out2.write(label + '\n')


def preprocess(domain):
    print('\t' + domain + ' train set ...')
    preprocess_train(domain)
    # print('\t' + domain + ' test set ...')
    # preprocess_test(domain)


if __name__ == "__main__":
    print('Preprocessing raw review sentences ...')
    preprocess('ethicsbot')

Preprocessing raw review sentences ...
	ethicsbot train set ...


In [79]:
#!/usr/bin/env python
#  -*- coding: utf-8  -*-

import gensim
import codecs


class Sentences(object):
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in codecs.open(self.filename, 'r', 'utf-8'):
            yield line.split()


def main(domain):
    source = 'preprocessed_data/%s/train.txt' % domain
    model_file = 'preprocessed_data/%s/w2v_embedding' % domain
    sentences = Sentences(source)
    print("m1")
    model = gensim.models.Word2Vec(sentences, size=200, window=5, min_count=10, workers=4, sg=1, iter=2)
    print("m2")
    model.save(model_file)
    print("m3")


if __name__ == "__main__":
    print('Pre-training word embeddings ...')
    main('ethicsbot')

Pre-training word embeddings ...
m1
m2


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


m3


# Topic Model Training

In [6]:
!pip install numpy==1.17

     |████████████████████████████████| 20.4MB 1.3MB/s 
ERROR: xarray 0.15.1 has requirement setuptools>=41.2, but you'll have setuptools 39.1.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.15.4
    Uninstalling numpy-1.15.4:
      Successfully uninstalled numpy-1.15.4


In [0]:
# check env
#!pip list

In [1]:
cd /content/drive/My Drive/111 Ethicsbot/topic_modeling/Unsupervised-Aspect-Extraction/code

/content/drive/My Drive/111 Ethicsbot/topic_modeling/Unsupervised-Aspect-Extraction/code


In [2]:
!python3 train.py \
--emb ../preprocessed_data/restaurant/w2v_embedding \
--epochs 5 \
--domain restaurant \
-o ../pre_trained_model/ethicsbot \

2020-05-28 14:52:42,189 INFO Arguments:
2020-05-28 14:52:42,190 INFO   algorithm: adam
2020-05-28 14:52:42,190 INFO   aspect_size: 14
2020-05-28 14:52:42,190 INFO   batch_size: 50
2020-05-28 14:52:42,190 INFO   command: train.py --emb ../preprocessed_data/restaurant/w2v_embedding --epochs 5 --domain restaurant -o ../pre_trained_model/ethicsbot
2020-05-28 14:52:42,190 INFO   domain: restaurant
2020-05-28 14:52:42,190 INFO   emb_dim: 200
2020-05-28 14:52:42,190 INFO   emb_path: ../preprocessed_data/restaurant/w2v_embedding
2020-05-28 14:52:42,190 INFO   epochs: 5
2020-05-28 14:52:42,190 INFO   maxlen: 0
2020-05-28 14:52:42,190 INFO   neg_size: 20
2020-05-28 14:52:42,191 INFO   ortho_reg: 0.1
2020-05-28 14:52:42,191 INFO   out_dir_path: ../pre_trained_model/ethicsbot
2020-05-28 14:52:42,191 INFO   seed: 1234
2020-05-28 14:52:42,191 INFO   vocab_size: 9000
Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (typ